In [1]:
%cd ..
%env TORCH_CUDNN_SDPA_ENABLED=1

/workspace/shai_hulud/ppc_experiments
env: TORCH_CUDNN_SDPA_ENABLED=1


In [2]:
import argparse
import collections
import numpy as np
import pyro
import torch
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
import trainer.trainer as module_trainer

In [3]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [4]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [5]:
from utils import read_json

config = read_json("experiments/ppc_flowers_config.json")
config = ConfigParser(config)

In [6]:
logger = config.get_logger('train')

# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

# build model architecture, then print to console
model = config.init_obj('arch', module_arch)
logger.info(model)

# get function handles of metrics
metrics = [getattr(module_metric, met) for met in config['metrics']]

# build optimizer.
if "lr_scheduler" in config:
    lr_scheduler = getattr(pyro.optim, config["lr_scheduler"]["type"])
    lr_scheduler = optimizer = lr_scheduler({
        "optimizer": getattr(torch.optim, config["optimizer"]["type"]),
        "optim_args": config["optimizer"]["args"]["optim_args"],
        **config["lr_scheduler"]["args"]
    })
else:
    optimizer = config.init_obj('optimizer', pyro.optim)
    lr_scheduler = None

# build trainer
# kwargs = config['trainer'].pop('args')
trainer = config.init_obj('trainer', module_trainer, model, metrics, optimizer,
                          config=config, data_loader=data_loader,
                          valid_data_loader=valid_data_loader,
                          lr_scheduler=lr_scheduler)

DiffusionPpc(
  (diffusion): DiffusionStep(
    (unet): ScoreNetwork0(
      (_convs): ModuleList(
        (0): Sequential(
          (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): LogSigmoid()
        )
        (1): Sequential(
          (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (2): LogSigmoid()
        )
        (2): Sequential(
          (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (2): LogSigmoid()
        )
        (3): Sequential(
          (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (2): LogSigmoid()
        )
        (4): Sequential(
          (0): MaxPool2d(ker

In [7]:
logger.info(trainer.config.log_dir)

saved/log/FlowersDiffusion_Ppc/0502_011314


In [8]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Train Epoch: 1 [0/918 (0%)] Loss: 75280016.000000
Train Epoch: 1 [16/918 (2%)] Loss: 65736224.000000
Train Epoch: 1 [32/918 (3%)] Loss: 68942752.000000
Train Epoch: 1 [48/918 (5%)] Loss: 66970356.000000
Train Epoch: 1 [64/918 (7%)] Loss: 76079696.000000
Train Epoch: 1 [80/918 (9%)] Loss: 69918080.000000
Train Epoch: 1 [96/918 (10%)] Loss: 62746536.000000
Train Epoch: 1 [112/918 (12%)] Loss: 56976864.000000
Train Epoch: 1 [128/918 (14%)] Loss: 75223232.000000
Train Epoch: 1 [144/918 (16%)] Loss: 67858048.000000
Train Epoch: 1 [160/918 (17%)] Loss: 59077404.000000
Train Epoch: 1 [176/918 (19%)] Loss: 69545896.000000
Train Epoch: 1 [192/918 (21%)] Loss: 55390720.000000
Train Epoch: 1 [208/918 (23%)] Loss: 61662688.000000
Train Epoch: 1 [224/918 (24%)] Loss: 76061824.000000
Train Epoch: 1 [240/918 (26%)] Loss: 58553076.000000
Train Epoch: 1 [256/918 (28%)] Loss: 63761920.000000
Train Epoch: 1 [272/918 (30%)] Loss: 66139144.000000
Train Epoch: 1 [288/918 (31%)] Loss: 58706080.000000
Train E

KeyboardInterrupt: 

In [ ]:
trainer.model.eval()
trainer.model.cpu()
trainer.cpu()
trainer.train_particles.cpu()
trainer.valid_particles.cpu()

In [ ]:
trainer.model.graph.clear()
trainer._load_particles(range(trainer.data_loader.batch_size), False)

In [ ]:
for site in trainer.model.graph.nodes:
    trainer.model.graph.nodes[site]['is_observed'] = trainer.model.graph.nodes[site]['value'] is not None

In [ ]:
import utils

In [ ]:
with pyro.plate_stack("forward", (trainer.num_particles, trainer.data_loader.batch_size)):
    model = pyro.condition(trainer.model, data={k: v['value'] for k, v in trainer.model.graph.nodes.items()})
    xs = model()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(10):
    plt.imshow(xs.mean(dim=0)[i].squeeze().detach().cpu().numpy())
    plt.show()